# Análise de sentimento
Aqui vou fazer um script para calcular a análise de sentimento das reviews

In [26]:
import numpy as np
import pandas as pd
from transformers import pipeline
from tqdm import tqdm
import hashlib
import math

In [48]:
rows = 20
chunk = 1000
chunk_pipeline = 1000
file = 'dados/reviewsTestToronto'

In [4]:
sentiment_pipeline = pipeline("text-classification", model="nlptown/bert-base-multilingual-uncased-sentiment")

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

C:\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Maria Haru\.cache\huggingface\hub\models--nlptown--bert-base-multilingual-uncased-sentiment. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


In [54]:
def reset_file():
    pd.DataFrame([], columns=['label', 'score', 'business_id', 'user_id', 'text_hash', 'useful', 'cool', 'funny']).to_csv(file + 'Sentiment.csv', mode='w', header=True, index=False)

def proccess(df_chunk):
    results = []
    #for df_chunk in tqdm(np.array_split(df, chunk_pipeline), total=len(df)/chunk_pipeline):
    obj_list = sentiment_pipeline(list(df_chunk['text'].apply(lambda x: x[0:512])))
    for obj, items in zip(obj_list, df_chunk.iterrows()):
        row = items[1]
        obj['label'] = int(obj['label'].split(' ')[0])
        obj['score'] = float(obj['score'])
        obj['business_id'] = row['business_id']
        obj['user_id'] = row['user_id']
        obj['text_hash'] = hashlib.sha1(row['text'].encode()).hexdigest()
        obj['useful'] = row['useful']
        obj['cool'] = row['cool']
        obj['funny'] = row['funny']
        results.append(obj)
    
    save_df = pd.DataFrame.from_dict(results)
    results = []
    save_df.to_csv(file + 'Sentiment.csv', mode='a', header=False, index=False)



In [55]:
reset_file()
for df in tqdm(pd.read_csv(file + '.csv', chunksize=chunk), total=math.ceil(34474 / chunk)):
    proccess(df)

100%|██████████| 35/35 [07:22<00:00, 12.63s/it]


In [50]:
df = pd.read_csv(file +'.csv')
df

,user_id,business_id,useful,funny,cool,text,date
0,5eogH-lsoc30dyK9doJMRw,-3pJJf-vHQfysafL0r8YsA,0,0,0,"I always come here, mainly because it's a 5 mi...",2017-06-29 03:47:19
1,Z9dE9GzseN40Ld9BAbIYlg,-3pJJf-vHQfysafL0r8YsA,0,0,0,I have been observing for some times. I don't ...,2018-05-14 22:12:13
2,-QdEn8wCiRE4XHvmTCPITw,-3pJJf-vHQfysafL0r8YsA,4,0,1,Typical Asian market. Great prices on exotic f...,2015-06-20 19:21:05
3,VCvtbcxEl6WxK2x0zGYkcQ,-3pJJf-vHQfysafL0r8YsA,0,0,0,Very very poor customer service.Old Dutch blea...,2018-11-13 15:07:22
4,-RCD8F7qbsLfzT3k1HtMxg,-5Np0EVLRVqjKlRwf_GShA,0,0,0,Fast and efficient dry cleaning service right ...,2015-01-18 18:26:07
...,...,...,...,...,...,...,...
34469,alUuOskFSl1bODjnce2PpQ,zrQYQHawhaolF7muiU6Ckw,4,0,2,"So, I went home and checked my receipt and rea...",2010-11-10 22:56:56
34470,oiBG_9xY4E4w4JWTCQR9sw,zrQYQHawhaolF7muiU6Ckw,2,1,1,Came in to the store to find a strapless bra a...,2016-12-15 21:55:09
34471,3uWCWMWcrn8YSH_qBvlr6w,zrQYQHawhaolF7muiU6Ckw,6,4,4,This place looks like nothing else in the Eato...,2010-11-04 14:59:00
34472,LtVQD4Axwr21Aw2eR-tkJg,zrQYQHawhaolF7muiU6Ckw,1,0,0,"Son secret? ses prix, toujours plus haut!\n\nB...",2012-01-30 23:31:57


In [32]:
df = pd.read_csv('dados/reviewsTrainToronto.csv', nrows=10)
df

,user_id,business_id,useful,funny,cool,text,date
0,Y2TBSd3ExydbgEnVxAt_QA,--DaPTJW3-tB1vP-PfdTEg,0,0,0,My sister and in laws were in town and we want...,2018-04-17 00:19:12
1,Rp-cSUHm-EKBxgBS73PNNg,--DaPTJW3-tB1vP-PfdTEg,3,0,2,We really enjoy this breakfast restaurant in t...,2016-03-19 03:52:08
2,2fXZuNKP7Bo-yRrINVbVVA,--DaPTJW3-tB1vP-PfdTEg,0,0,0,"Just tried this breakfast joint, didn't want t...",2013-09-07 22:20:08
3,fn3Wim-2j042IEDe6VhJFQ,--DaPTJW3-tB1vP-PfdTEg,5,1,5,Stopped in on a weekday while in Toronto just ...,2018-05-10 18:33:34
4,CD9dSllum_L_OvpRW0YH0w,--DaPTJW3-tB1vP-PfdTEg,0,0,0,Not great. Ordered the eggs florentine with ho...,2018-01-11 19:57:49
5,Q9mA60HnY87C1TW5kjAZ6Q,--DaPTJW3-tB1vP-PfdTEg,2,0,1,This place lives somewhere in between diner di...,2014-07-30 00:38:57
6,TTCzwn_6k1AwSuqAB8oLBA,--DaPTJW3-tB1vP-PfdTEg,0,0,0,Awesome place. Love the liberal use of corned ...,2013-08-16 02:12:01
7,ILasiYMdzxhUf9Mkcl8EnQ,--DaPTJW3-tB1vP-PfdTEg,0,0,0,Excellent and friendly service! Nice clean re...,2012-11-11 18:29:56
8,DTfe51_9B98LevKahtdgXQ,--DaPTJW3-tB1vP-PfdTEg,0,0,0,Completely mediocre. I was there on a Saturday...,2014-03-01 22:49:13
9,a_adronfoaxpphBD5d_3Cw,--DaPTJW3-tB1vP-PfdTEg,0,0,0,Sunnyside grill is sort of american diner sort...,2016-08-04 22:21:52


In [30]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/guilherme/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

100%|███████████████████████████████████████████| 10/10 [00:03<00:00,  3.15it/s]

saved 9


[]

Index(['user_id', 'business_id', 'useful', 'funny', 'cool', 'text', 'date'], dtype='object')

In [78]:
pd.concat([df.iloc[begining_index:index+1], pd.DataFrame.from_dict(results)], axis=1)

,user_id,business_id,useful,funny,cool,text,date,label,score
0,Y2TBSd3ExydbgEnVxAt_QA,--DaPTJW3-tB1vP-PfdTEg,0,0,0,My sister and in laws were in town and we want...,2018-04-17 00:19:12,5 stars,0.533591
1,Rp-cSUHm-EKBxgBS73PNNg,--DaPTJW3-tB1vP-PfdTEg,3,0,2,We really enjoy this breakfast restaurant in t...,2016-03-19 03:52:08,4 stars,0.697149
2,2fXZuNKP7Bo-yRrINVbVVA,--DaPTJW3-tB1vP-PfdTEg,0,0,0,"Just tried this breakfast joint, didn't want t...",2013-09-07 22:20:08,4 stars,0.315594
3,fn3Wim-2j042IEDe6VhJFQ,--DaPTJW3-tB1vP-PfdTEg,5,1,5,Stopped in on a weekday while in Toronto just ...,2018-05-10 18:33:34,4 stars,0.484586
4,CD9dSllum_L_OvpRW0YH0w,--DaPTJW3-tB1vP-PfdTEg,0,0,0,Not great. Ordered the eggs florentine with ho...,2018-01-11 19:57:49,2 stars,0.646709
5,Q9mA60HnY87C1TW5kjAZ6Q,--DaPTJW3-tB1vP-PfdTEg,2,0,1,This place lives somewhere in between diner di...,2014-07-30 00:38:57,4 stars,0.656242
6,TTCzwn_6k1AwSuqAB8oLBA,--DaPTJW3-tB1vP-PfdTEg,0,0,0,Awesome place. Love the liberal use of corned ...,2013-08-16 02:12:01,5 stars,0.839217
7,ILasiYMdzxhUf9Mkcl8EnQ,--DaPTJW3-tB1vP-PfdTEg,0,0,0,Excellent and friendly service! Nice clean re...,2012-11-11 18:29:56,5 stars,0.812308
8,DTfe51_9B98LevKahtdgXQ,--DaPTJW3-tB1vP-PfdTEg,0,0,0,Completely mediocre. I was there on a Saturday...,2014-03-01 22:49:13,2 stars,0.690526
9,a_adronfoaxpphBD5d_3Cw,--DaPTJW3-tB1vP-PfdTEg,0,0,0,Sunnyside grill is sort of american diner sort...,2016-08-04 22:21:52,4 stars,0.548095


In [54]:
pd.DataFrame(row).transpose()

,user_id,business_id,useful,funny,cool,text,date
0,Y2TBSd3ExydbgEnVxAt_QA,--DaPTJW3-tB1vP-PfdTEg,0,0,0,My sister and in laws were in town and we want...,2018-04-17 00:19:12


In [45]:
result_df

,label,score
0,5 stars,0.533591


In [43]:
df['text'][0]

'My sister and in laws were in town and we wanted a central meeting point. My sister wanted more of a basic breakfast and we decided on here. We are so glad that we did. Great food, good service and good prices. They only have drip coffee (if you are a coffee snob) but I am told it is good. It is small, tables of 2 or 4. We had 5 so we had to wait about 15 minutes for a table which was fine. We got to check out the food as it came out of the kitchen. The home fries were awesome, the eggs were cooked to order and I will be back.'

In [42]:
sentiment_pipeline(df['text'][0])

[{'label': '5 stars', 'score': 0.5335907340049744}]